# distilGPT2 Fine-Tuning v2

In [2]:
import torch
from tqdm.notebook import tqdm
tqdm.pandas()
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('dark_background')

ver = 2

## Load Data

In [3]:
import pandas as pd
data = pd.read_csv('../out.csv')
data

,Unnamed: 0,lastfm_url,track,artist,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,spotify_id,genre,Lyric,language
0,1,https://www.last.fm/music/metallica/_/st.%2banger,St. Anger,Metallica,['aggressive'],8,3.710000,5.833000,5.427250,727a2529-7ee8-4860-aef6-7959884895cb,3fOc9x06lKJBhz435mInlH,metal,Saint Anger 'round my neck\nSaint Anger 'round...,en
1,3,https://www.last.fm/music/m.i.a./_/bamboo%2bbanga,Bamboo Banga,M.I.A.,"['aggressive', 'fun', 'sexy', 'energetic']",13,6.555071,5.537214,5.691357,99dd2c8c-e7c1-413e-8ea4-4497a00ffa18,6tqFC1DIOphJkCwrjVzPmg,hip-hop,"Road runner, road runner\nGoing hundred mile p...",en
2,5,https://www.last.fm/music/drowning%2bpool/_/st...,Step Up,Drowning Pool,['aggressive'],9,2.971389,5.537500,4.726389,49e7b4d2-3772-4301-ba25-3cc46ceb342e,4Q1w4Ryyi8KNxxaFlOQClK,metal,Come!\n\nIf our own lives aren’t directly affe...,en
3,11,https://www.last.fm/music/kanye%2bwest/_/feedback,Feedback,Kanye West,['aggressive'],1,3.080000,5.870000,5.490000,NaN,49fT6owWuknekShh9utsjv,hip-hop,"Ayy, y'all heard about the good news?\nY'all s...",en
4,13,https://www.last.fm/music/deftones/_/7%2bwords,7 Words,Deftones,"['aggressive', 'angry']",10,3.807121,5.473939,4.729091,1a826083-5585-445f-a708-415dc90aa050,6DoXuH326aAYEN8CnlLmhP,nu metal,"I'll never be the same, breaking decency\nDon'...",en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16792,229432,https://www.last.fm/music/noblegases/_/xenon,Xenon,NobleGases,['noble'],2,6.160000,3.695000,6.130000,NaN,1AePjgLLtzF0abbfcgYdLI,chill,You're floating out astray\nThis cold and life...,en
16793,229435,https://www.last.fm/music/kurt%2bvile/_/wild%2...,Wild Imagination,Kurt Vile,['transparent'],2,6.925000,4.975000,6.190000,NaN,1Gn0oYQiQHp7KF4DcR2g4t,NaN,I'm looking at you\nBut It's only a picture so...,en
16794,229436,https://www.last.fm/music/portugal.%2bthe%2bma...,Oh Lord,Portugal. The Man,['transparent'],1,5.370000,3.450000,5.330000,7ea228f9-16d0-474d-8c51-5a1a9810ddde,6YG8cjbrjhDhlYMiQnibUD,indie,\n\n\nWhere do I fit in\nI am waiting here for...,en
16795,229443,https://www.last.fm/music/porcelain%2band%2bth...,Transparent,Porcelain and The Tramps,['transparent'],3,6.613333,4.633333,5.773333,NaN,NaN,industrial,Wish I were transparent\nYou could see right t...,en


## Load Model & Tokenizer

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [7]:
model = AutoModelForCausalLM.from_pretrained('distilgpt2')  # Use this to load default distilgpt2

In [8]:
model = AutoModelForCausalLM.from_pretrained(f'./v{ver}_final_model')  # Use this to load our fine tuned model

## Preprocess Tags

Need to run this

In [9]:
data['seeds'] = data['seeds'].apply(lambda i: eval(i))

## Preprocess Lyrics

In [10]:
chunk_size = 128
buffer = 8
question = 'What are the moods evoked by the following song excerpt?'

def text_repr(lyrics: str, tags: list) -> list[str]:
    tags = tags[:10]  # Trim the number of tags to prevent some errors
    extra_tokens = len(tokenizer(f'Q: {question}\n\nA: {", ".join(tags)}{tokenizer.eos_token}')['input_ids'])
    this_chunk_size = chunk_size - extra_tokens
    all_tokens = tokenizer(lyrics)['input_ids']
    chunks = [all_tokens[i:i + this_chunk_size] for i in range(0, len(all_tokens), this_chunk_size)]
    chunks = [f'Q: {question}\n{tokenizer.decode(i)}\nA: {", ".join(tags)}{tokenizer.eos_token}' for i in chunks]
    chunks = tokenizer(chunks, padding=True, truncation=True, max_length=chunk_size + buffer)['input_ids']
    return chunks[:-1]  # Remove last element to avoid misaligned sequences

TypeError: 'type' object is not subscriptable

This code creates a training dataset with all lyrics + tags.

In [11]:
prompts = data['Lyric']
tags = data['seeds']

tokenized_data_expanded = [text_repr(prompts.iloc[i], tags.iloc[i]) for i in tqdm(range(len(prompts)))]
tokenized_data = []
for i in tokenized_data_expanded:
    tokenized_data += i

  0%|          | 0/16797 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1083 > 1024). Running this sequence through the model will result in indexing errors


In [12]:
tokenized_data = [i for i in tokenized_data if len(i) == chunk_size]  # Remove any data which still isn't the right size

## Build Dataset

In [13]:
from sklearn.model_selection import train_test_split

x_train, x_val = train_test_split(tokenized_data, test_size=.1)

ModuleNotFoundError: No module named 'sklearn'

In [14]:
print(tokenizer.decode(x_train[0]))

NameError: name 'x_train' is not defined

## Train Model

No need to do this since fine tuning is already complete.

In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir=f"distilgpt2-finetuned-v{ver}",
    save_total_limit=10,
    evaluation_strategy="epoch",
    # evaluation_strategy="steps",
    # eval_steps=10,
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=15
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=x_train,
    eval_dataset=x_val,
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,2.920100,2.845643
2,2.847500,2.806225
3,2.820100,2.782271
4,2.754400,2.765009
5,2.725600,2.752496
6,2.704000,2.743229
7,2.713300,2.739249
8,2.665500,2.732555
9,2.646800,2.729696
10,2.635600,2.726301


TrainOutput(global_step=77235, training_loss=2.7027885776326412, metrics={'train_runtime': 69264.2248, 'train_samples_per_second': 8.919, 'train_steps_per_second': 1.115, 'total_flos': 2.01778250121216e+16, 'train_loss': 2.7027885776326412, 'epoch': 15.0})

In [ ]:
trainer.save_model(f'v{ver}_final_model')  # DO NOT RUN THIS

## Evaluation

In [21]:
from transformers import pipeline
text_generator = pipeline('text-generation', tokenizer=tokenizer, model=model)

In [22]:
chunk_size = 128
buffer = 8
question = 'What are the moods evoked by the following song excerpt?'

def process_lyrics(lyrics: str) -> list[str]:
    extra_tokens = len(tokenizer(f'Q: {question}\n\nA: ')['input_ids'])
    this_chunk_size = chunk_size - extra_tokens
    all_tokens = tokenizer(lyrics)['input_ids']
    chunks = [all_tokens[i:i + this_chunk_size] for i in range(0, len(all_tokens), this_chunk_size)]
    chunks = [f'Q: {question}\n{tokenizer.decode(i)}\nA: ' for i in chunks]
    chunks = tokenizer(chunks, padding=True, truncation=True, max_length=chunk_size + buffer)['input_ids']
    return chunks  # Remove last element to avoid misaligned sequences

In [23]:
LYRICS = """
Whores in this house
There's some whores in this house
There's some whores in this house
There's some whores in this house (Hol' up)
I said certified freak, seven days a week
Wet-ass pussy, make that pullout game weak, woo (Ah)

Yeah, yeah, yeah, yeah
Yeah, you fuckin' with some wet-ass pussy
Bring a bucket and a mop for this wet-ass pussy
Give me everything you got for this wet-ass pussy

Beat it up, nigga, catch a charge
Extra large and extra hard
Put this pussy right in your face
Swipe your nose like a credit card
Hop on top, I wanna ride
I do a kegel while it's inside
Spit in my mouth, look in my eyes
This pussy is wet, come take a dive
Tie me up like I'm surprised
Let's roleplay, I'll wear a disguise
I want you to park that big Mack truck right in this little garage
Make it cream, make me scream
Out in public, make a scene
I don't cook, I don't clean
But let me tell you how I got this ring (Ayy, ayy)

Gobble me, swallow me, drip down the side of me (Yeah)
Quick, jump out 'fore you let it get inside of me (Yeah)
I tell him where to put it, never tell him where I'm 'bout to be (Huh)
I'll run down on him 'fore I have a nigga runnin' me (Pow, pow, pow)
Talk your shit, bite your lip (Yeah)
Ask for a car while you ride that dick (While you ride that dick)
You really ain't never gotta fuck him for a thang (Yeah)
He already made his mind up 'fore he came (Ayy, ah)
Now get your boots and your coat for this wet-ass pussy (Ah, ah, ah)
He bought a phone just for pictures of this wet-ass pussy (Click, click, click)
Paid my tuition just to kiss me on this wet-ass pussy (Mwah, mwah, mwah)
Now make it rain if you wanna see some wet-ass pussy (Yeah, yeah)

Look, I need a hard hitter, need a deep stroker
Need a Henny drinker, need a weed smoker
Not a garter snake, I need a king cobra
With a hook in it, hope it lean over
He got some money, then that's where I'm headed
Pussy A1 just like his credit
He got a beard, well, I'm tryna wet it
I let him taste it, now he diabetic
I don't wanna spit, I wanna gulp
I wanna gag, I wanna choke
I want you to touch that lil' dangly thing that swing in the back of my throat
My head game is fire, punani Dasani
It's goin' in dry and it's comin' out soggy
I ride on that thing like the cops is behind me (Yeah, ah)
I spit on his mic and now he tryna sign me, woo

Your honor, I'm a freak bitch, handcuffs, leashes
Switch my wig, make him feel like he cheatin'
Put him on his knees, give him somethin' to believe in
Never lost a fight, but I'm lookin' for a beatin' (Ah)
In the food chain, I'm the one that eat ya
If he ate my ass, he's a bottom-feeder
Big D stand for big demeanor
I could make ya bust before I ever meet ya
If it don't hang, then he can't bang
You can't hurt my feelings, but I like pain
If he fuck me and ask "Whose is it?"
When I ride the dick, I'ma spell my name, ah

Yeah, yeah, yeah
Yeah, you fuckin' with some wet-ass pussy
Bring a bucket and a mop for this wet-ass pussy
Give me everything you got for this wet-ass pussy
Now from the top, make it drop, that's some wet-ass pussy
Now get a bucket and a mop, that's some wet-ass pussy
I'm talkin' wap, wap, wap, that's some wet-ass pussy
Macaroni in a pot, that's some wet-ass pussy, huh

There's some whores in this house
There's some whores in this house
There's some whores in this house
There's some whores in this house
There's some whores in this house
There's some whores in this house
There's some whores in this house
There's some whores in this house
There's some whores in this house
There's some whores in this house
"""

In [24]:
processed_lyrics_chunks = process_lyrics(LYRICS)
len(processed_lyrics_chunks)

10

In [25]:
idx = 1
print(text_generator(tokenizer.decode(processed_lyrics_chunks[idx]), 
                     max_length=len(processed_lyrics_chunks[idx]) + 10)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: What are the moods evoked by the following song excerpt?
 pussy
Give me everything you got for this wet-ass pussy

Beat it up, nigga, catch a charge
Extra large and extra hard
Put this pussy right in your face
Swipe your nose like a credit card
Hop on top, I wanna ride
I do a kegel while it's inside
Spit in my mouth, look in my eyes
This pussy is wet, come take a dive
Tie me up like I'm surprised
Let's roleplay, I'll wear a disguise

A: ive, energetic, energetic, energetic, energetic,


1. function to analyze an entire song
    - collate tags from all excerpts
    - rank tags
    - select some number of best tags as output
2. function to evaluate model's performance on a single song
    - inputs: model's generated text, correct tags from dataset ground truth
    - output: numerical score indicating how well the model performed (ideally 0-1)
    - could factor in multiple runs per song?
3. apply function to all songs in dataset and plot scores

In [26]:
tokenizer.SPECIAL_TOKENS_ATTRIBUTES

['bos_token',
 'eos_token',
 'unk_token',
 'sep_token',
 'pad_token',
 'cls_token',
 'mask_token',
 'additional_special_tokens']

In [32]:
import sys
print(sys.executable)
!pip install pyspellchecker

/Users/sarah/miniconda3/envs/torch/bin/python
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Using cached pyspellchecker-0.7.1-py3-none-any.whl (2.5 MB)


In [37]:
PAD_TOKEN = tokenizer(tokenizer.pad_token)['input_ids'][0]

#def is_word(word: str) -> bool:
from spellchecker import SpellChecker
spell = SpellChecker()
def check(word: str) -> bool:
    if word == spell.correction(word) and word != 'ive':
        return True
    else:
        return False
    
 # TODO: IMPLEMENT THIS

def tags_for_excerpt(tokens: list) -> str:
    # tokens = tokenizer(excerpt)['input_ids']
    excerpt = tokenizer.decode(tokens)
    output = text_generator(excerpt, 
                         max_length=len(tokens) + 10)[0]['generated_text']
    response = output[output.rfind('A: ') + 3:]
    response_tags = response.split(', ')
    response_tags = [i.strip(', \nÂ') for i in response_tags]
    response_tags = list(filter(is_word, response_tags))
    return response_tags

def tags_for_all_lyrics(lyrics: str) -> list[list[str]]:
    chunks = process_lyrics(lyrics)
    result = []
    for i in chunks[:-1]:
        result.append(tags_for_excerpt(i))
    result.append(tags_for_excerpt(chunks[-1][:chunks[-1].index(PAD_TOKEN)]))
    return result

tags = tags_for_all_lyrics(LYRICS)
tags

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[['', 'bravado', 'confident', 'raucous'],
 ['ive', 'smooth', 'driving', 'atmospheric', 'calm'],
 ['ive', 'sweet', 'mellow', 'soft', 'quiet'],
 ['iced', 'energetic', 'energetic', 'exciting', 'rousing'],
 ['manic', 'calm', 'calm', 'quiet', 'sweet'],
 ['iced', 'wistful', 'lush', 'narrative'],
 ['', 'aggressive', 'fierce', 'angry', 'strong'],
 ['ersatz', 'witty', 'detached', 'humorous', 'whims'],
 ['ileasy', 'brash', 'thrilling', 'triumphant'],
 ['strong', 'driving', 'sexy', 'sexy', 'sweet']]

In [38]:
only_words = [[j for j in i if check(j)] for i in tags]
only_words

[['bravado', 'confident', 'raucous'],
 ['smooth', 'driving', 'atmospheric', 'calm'],
 ['sweet', 'mellow', 'soft', 'quiet'],
 ['iced', 'energetic', 'energetic', 'exciting', 'rousing'],
 ['manic', 'calm', 'calm', 'quiet', 'sweet'],
 ['iced', 'wistful', 'lush', 'narrative'],
 ['aggressive', 'fierce', 'angry', 'strong'],
 ['ersatz', 'witty', 'detached', 'humorous', 'whims'],
 ['brash', 'thrilling', 'triumphant'],
 ['strong', 'driving', 'sexy', 'sexy', 'sweet']]

Let's say we make a function song_analysis_pipeline(song_lyrics, num_tags_to_select).

For each dataset element:

- Lyrics
- Correct tags

Count correct tags, results = song_analysis_pipeline(Lyrics, len(correct_tags)).

score = number of tags in results which are also in correct_tags / total number of correct tags